<img src="https://ai.google.dev/static/mediapipe/images/solutions/pose_landmarks_index.png" style="height:300px">
0 - nose
1 - left eye (inner)
2 - left eye
3 - left eye (outer)
4 - right eye (inner)
5 - right eye
6 - right eye (outer)
7 - left ear
8 - right ear
9 - mouth (left)
10 - mouth (right)
11 - left shoulder
12 - right shoulder
13 - left elbow
14 - right elbow
15 - left wrist
16 - right wrist
17 - left pinky
18 - right pinky
19 - left index
20 - right index
21 - left thumb
22 - right thumb
23 - left hip
24 - right hip
25 - left knee
26 - right knee
27 - left ankle
28 - right ankle
29 - left heel
30 - right heel
31 - left foot index
32 - right foot index

In [ ]:
# create meadiapipe instance
with mp_pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence = 0.5) as pose:
    cap = VideoCapture(0)
    frame_width = cap.get(CAP_PROP_FRAME_WIDTH)
    frame_height = cap.get(CAP_PROP_FRAME_HEIGHT)
    print(frame_width, frame_height)
    while(cap.isOpened()):
        success,frame = cap.read()
        frame = cv2.flip(frame, 1) # flip it for webcams
        image = cvtColor(frame,COLOR_BGR2RGB)
        image.flags.writeable = False
        a=2
        result = pose.process(image)
        
        image.flags.writeable = True
        image = cvtColor(image,COLOR_RGB2BGR)
        #EXTRACT LANDMARKS
        try:
            landmarks = result.pose_landmarks.landmark
            left_upper_back = np.array([landmarks[11].x,landmarks[11].y])
            left_lower_back = np.array([landmarks[23].x,landmarks[23].y])
            left_knee = np.array([landmarks[25].x,landmarks[25].y])
            left_ankle = np.array([landmarks[27].x,landmarks[27].y])
            left_upper_back_pixel = np.multiply(left_upper_back, [frame_width, frame_height])
            left_lower_back_pixel = np.multiply(left_lower_back, [frame_width, frame_height])
            if(166<calculate_joint_angle(j1=left_lower_back,j2=left_knee,j3=left_ankle)<180):
                initial_back_length = np.linalg.norm(left_upper_back_pixel-left_lower_back_pixel)
                cv2.putText(image,str(initial_back_length),
                        tuple(np.multiply(left_upper_back,[frame_width,frame_height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
                        )
            back_slacking(left_upper_back_pixel, left_lower_back_pixel, initial_back_length, image, frame_width, frame_height)
            kneeOverToes(left_lower_back,left_knee,left_ankle)
            left_heel = np.array([landmarks[29].x,landmarks[29].y])
            left_foot_index = np.array([landmarks[31].x,landmarks[31].y])
            heelsOffGround(left_heel,left_foot_index)
            cv2.putText(image,str(calculate_joint_angle(j1=left_lower_back,j2=left_knee,j3=left_ankle)),
                        tuple(np.multiply(left_ankle,[frame_width,frame_height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
                        )
            
        except Exception as e:
            print(f"An error occurred: {e}")
            pass
        
        #render detections
        
        mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                  )
        
        
        imshow('Image from WebCam',image)
        if(waitKey(25)==27):
            break
    cap.release()
    destroyAllWindows()

In [13]:
# lets prepare for video input
import mediapipe as mp
from mediapipe import solutions
import numpy as np
import cv2
from cv2 import VideoCapture, waitKey,imshow,CAP_PROP_FRAME_WIDTH, CAP_PROP_FRAME_HEIGHT,CAP_PROP_FPS,destroyAllWindows,COLOR_BGR2GRAY,COLOR_RGB2BGR,COLOR_BGR2RGB,cvtColor
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose

class SquatAnalyser():
    def __init__(self,*,mode:int):
        '''
            mode 0 -> Inbuilt Webcam
            mode 1 -> Video File
        '''
        if(mode == 0):
            self.cap = VideoCapture('/home/rohan/fitnessTrainer/squat.mp4')
        else:
            with open("addPathToFileHere.txt","r") as f:
                try:
                    self.capture = VideoCapture(f.readline())
                except Exception as e:
                    print("Error while handling video file, make sure you have written correct file path. ",e)
        self.frame_width = self.cap.get(CAP_PROP_FRAME_WIDTH)
        self.frame_height = self.cap.get(CAP_PROP_FRAME_HEIGHT)
        self.joints = {} #store relevant joint coordinates
        self.reps = 0 # variable for counting repetitions
        self.initial_back_length = 0
        self.initial_heel_angle = 0
        self.stage = "up" #initial position of SQUAT. Will be set to "down" when user goes parallel or below parallel to ground
        
    def initialise_bounds(self,shoulder,hip,heel,foot_index):
        left_upper_back_pixel = np.multiply(shoulder, [self.frame_width, self.frame_height])
        left_lower_back_pixel = np.multiply(hip, [self.frame_width, self.frame_height])
        self.initial_back_length = np.linalg.norm(left_upper_back_pixel-left_lower_back_pixel) 
        self.initial_heel_angle =  np.abs(180*np.arctan2(heel[1]-foot_index[1],heel[0]-foot_index[0])/np.pi)      
          
    def calculate_joint_angle(self,*,j1,j2,j3):
        '''
            Calculates angle between j1 j2 and j3
        '''
        v1 = np.array(j1-j2)
        v2 = np.array(j3-j2)
        cos_angle = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
        radians = np.arccos(np.clip(cos_angle, -1, 1))
        angle = np.abs(radians*180.0/np.pi)
        if angle > 180:
            angle = 360 - angle
        return angle
    def back_slacking(self,upper_back, lower_back, initial_back_length, image, frame_width, frame_height):
        distance = np.linalg.norm(upper_back - lower_back)
        if distance + 23 < initial_back_length:
            return False
            
    def heels_off_ground(left_heel,left_foot_index,initial_heel_angle):
        radians = np.arctan2(left_heel[1]-left_foot_index[1],left_heel[0]-left_foot_index[0])
        angle = np.abs(radians*180/np.pi)
        return angle + 4 < initial_heel_angle
        
        
    def knee_over_toes(left_lower_back,left_knee,left_ankle):
        radians = np.arctan2(left_lower_back[1]-left_knee[1],left_lower_back[0]-left_knee[0])
        angle = np.abs(radians*180.0/np.pi)
        if(angle<40):
            return left_knee[0]>left_ankle[0]
        else:
            return True
        
    def ensure_proper_depth():
        pass
    def show_reps_on_screen(self,image,knee_hip_angle):
        if(knee_hip_angle<25 and self.stage=="up"):
            self.stage = "down"
        elif(knee_hip_angle>30 and self.stage=="down"):
            self.reps+=1
            self.stage= "up"
        cv2.putText(image,"Reps: "+str(self.reps),(10,10),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
    def draw_landmarks(self,image):
        #draw Circles on joints
        frame = [self.frame_width,self.frame_height]
        for joint in self.joints.values():
            cv2.circle(image,np.multiply(joint,frame).astype(int),3,(135, 53, 3),-1)
            cv2.circle(image,np.multiply(joint,frame).astype(int),6,(194, 99, 41),1)
            
        
        
        
        
        
    def create_mediapipe_instance(self):
        with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
            while(self.cap.isOpened()):
                ret,frame = self.cap.read()
                frame = cv2.flip(frame,1)
                image = cvtColor(frame,COLOR_BGR2RGB)
                image.flags.writeable = False

                result = pose.process(image)
                
                image.flags.writeable = True
                image = cvtColor(image,COLOR_RGB2BGR)
                try:
                    landmarks = result.pose_landmarks.landmark
                    #EXTRACT RELEVANT LANDMARKS
                    self.joints['shoulder'] = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y])
                    self.joints['hip'] = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
                    self.joints['knee'] = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
                    self.joints['ankle'] = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
                    self.joints['heel'] = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                                                    landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y])
                    self.joints['foot index'] = np.array([landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                                                    landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y])  
                    
                    self.draw_landmarks(image)
                    
                    if(166<self.calculate_joint_angle(j1 = self.joints['hip'],j2 = self.joints['knee'],j3 =self.joints['ankle'])<180):   
                        self.initialise_bounds(self.joints['shoulder'],self.joints['hip'],self.joints['heel'],self.joints['foot index'])
                    # check form
                    # if(self.back_slacking() or not self.knee_over_toes() or self.heels_off_ground()):
                    #     continue
                    
                    # Rep counter
                    knee_hip_angle =np.abs(180*np.arctan2(self.joints['hip'][1]-self.joints['knee'][1],self.joints['hip'][0]-self.joints['knee'][0])/np.pi)  
                    self.show_reps_on_screen(image,knee_hip_angle)

                    
                except Exception as e:
                    print("Maybe no Landmarks Detected ",e) 
                    pass 
                imshow('Image from WebCam',image)
                if(waitKey(25)==27):
                    break
            self.cap.release()
            destroyAllWindows()   
                        


In [14]:
sa = SquatAnalyser(mode=0)
sa.create_mediapipe_instance()

W0000 00:00:1721187825.148942   29093 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1721187825.186560   29093 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x84bad00) is not the object's thread (0x869a110).
Cannot move to target thread (0x84bad00)

QObject::moveToThread: Current thread (0x84bad00) is not the object's thread (0x869a110).
Cannot move to target thread (0x84bad00)

QObject::moveToThread: Current thread (0x84bad00) is not the object's thread (0x869a110).
Cannot move to target thread (0x84bad00)

QObject::moveToThread: Current thread (0x84bad00) is not the object's thread (0x869a110).
Cannot move to target thread (0x84bad00)

QObject::moveToThread: Current thread (0x84bad00) is not the object's thread (0x869a110).
Cannot move to tar